In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np
import os
from sklearn.model_selection import train_test_split
import sklearn.preprocessing

Using TensorFlow backend.


Fix random seed

In [2]:
seed = np.random.seed(7)

# Get back the convolutional part of a VGG network trained on ImageNet

In [3]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
model_vgg16_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

# Create your own input format (here 300x400x3)

In [4]:
input = Input(shape=(400,300,3),name = 'image_input')

# Use the generated model 

In [5]:
output_vgg16_conv = model_vgg16_conv(input)

# Add the fully-connected layers 

In [6]:
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(2, activation='softmax', name='predictions')(x)

# Create your own model 

In [7]:
my_model = Model(input=input, output=x)

/home/kevin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`
  """Entry point for launching an IPython kernel.


In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training

In [8]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 400, 300, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 55296)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 4096)              226496512 
_________________________________________________________________
fc2 (Dense)                  (None, 4096)              16781312  
_________________________________________________________________
predictions (Dense)          (None, 2)                 8194      
Total params: 258,000,706
Trainable params: 258,000,706
Non-trainable params: 0
______________________________________________________________

# Get data for train and test set

In [21]:
np_path = '/home/kevin/Desktop/OpenFoodFacts/data/npy'
imgs_np_path = os.path.join(np_path, 'images.npy')
labels_np_path = os.path.join(np_path, 'labels.npy')
Y = np.load(labels_np_path)
X = np.load(imgs_np_path)

Little work on Y

In [22]:
Y.shape = (2908,1)
b = Y=='1'
a = np.concatenate((b,~b), axis = 1)
Y = a

#a = Y.copy()
#label_binarizer = sklearn.preprocessing.LabelBinarizer()
#label_binarizer.fit(range(2))
#b = label_binarizer.transform(a)
#Y = b

In [23]:
print("The images data shape is {}.".format(X.shape))
print("The labels data shape is {}.".format(Y.shape))

The images data shape is (2908, 400, 300, 3).
The labels data shape is (2908, 2).


Now compile the model

In [ ]:
my_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Split train and test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=seed)

Remove the variables that are now uneseful but who takes lot of space in the RAM

In [ ]:
%reset_selective X, Y

# Learning phase: fit the model

In [ ]:
my_model.fit(X_train, y_train, validation_split=0.3, epochs=1, batch_size=10)

In [ ]:
scores = my_model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))